
- [x] transform the frequent patterns into json format
- [x] create output folders for patterns up to size 6 (vertices)
- [x] create output folders for patterns of size exactly 6 (vertices)
- [ ] for each frequency threshold and each random forest, find all occurrences of the frequent patterns in the random forest 

In [1]:
# create output directories
for dataset in adult wine-quality; do
    for variant in WithLeafEdges NoLeafEdges; do
        for patternsize in leq6 eq6; do
            mkdir forests/rootedFrequentTrees/${dataset}/${variant}/${patternsize}
        done
    done
done

In [3]:
# transform the frequent patterns from cString to json format
for dataset in adult wine-quality; do
    for variant in WithLeafEdges NoLeafEdges; do
        for f in forests/rootedFrequentTrees/${dataset}/${variant}/*.patterns; do
            ./cString2json.py leq 6 \
            < ${f} \
            > forests/rootedFrequentTrees/${dataset}/${variant}/leq6/`basename ${f} .patterns`.json
            
            ./cString2json.py eq 6 \
            < ${f} \
            > forests/rootedFrequentTrees/${dataset}/${variant}/eq6/`basename ${f} .patterns`.json
        done
    done
done

Traceback (most recent call last):
  File "./cString2json.py", line 103, in <module>
    result = parseCStringFileUpToSizePatterns(sys.stdin, int(sys.argv[2]))
  File "./cString2json.py", line 90, in parseCStringFileUpToSizePatterns
    jsonBlob = '[' + ',\n'.join(jsonCStrings) + ']'
  File "./cString2json.py", line 84, in <lambda>
    pairs = map(lambda t: (t[1], t[2]), tokens)
IndexError: list index out of range


In [ ]:
for dataset in adult wine-quality; do
    for transactionfile in `ls forests/${dataset}/text/*.json | grep -v DT`; do
        transactionBasename=`basename ${transactionfile} .json`        
        for variant in WithLeafEdges NoLeafEdges; do
            for patternsize in leq6 eq6; do
            
                echo processing ${transactionBasename} : ${variant} ${patternsize}
                for patternfile in forests/rootedFrequentTrees/${dataset}/${variant}/${patternsize}/${transactionBasename}*.json; do
                    transactionsWithEmbeddings=`basename ${patternfile} .json`_allEmbeddings.json
                    sem --id embeddings --jobs 3 \
                    ./findEmbeddings.py ${patternfile} ${transactionfile} \
                    > forests/rootedFrequentTrees/${dataset}/${variant}/${patternsize}/${transactionsWithEmbeddings}
                done
            done
        done
    done
done

sem --id embeddings --wait

processing ET_10 : WithLeafEdges leq6
processing ET_10 : WithLeafEdges eq6
processing ET_10 : NoLeafEdges leq6
processing ET_10 : NoLeafEdges eq6
processing ET_15 : WithLeafEdges leq6
processing ET_15 : WithLeafEdges eq6
processing ET_15 : NoLeafEdges leq6
